In [44]:
import yfinance as yf
from pandas import DataFrame
import pandas as pd
from datetime import date
import openai
from typing import Dict, Any
import os

openai.api_key = "sk-KYMTz3dbIpE9T46HklkZI4g1n89NeN1Xsp7zOtyK60T3BlbkFJmfEKBHcO_CRFrZJzhbT25xCIJFD1kehCARRTqyfXUA"

class LoadData:
    
    def __init__(self, data_prompt: str) -> None:
        
        # Transform the data from csv to a dataframe
        self.ticker_data = pd.read_csv("./tr4der/dataall_ticker_data.csv", header=0).transpose()
        self.ticker_data.columns = self.ticker_data.iloc[0]
        self.ticker_data = self.ticker_data.iloc[1:]
        self.ticker_data.reset_index(inplace=True)
        self.ticker_data.rename(columns={"index": "ticker"}, inplace=True)
        self.ticker_data['marketCap'] = self.ticker_data['marketCap'].astype(float)

        self.data_prompt = data_prompt
        self.pandas_code = self._pandas_code_generate(self.data_prompt)
        #self.filtered_dataframe = self._pandas_code_execute()
        
        
        
        
        
        # self.gpt_response = self._get_gpt_response()
        # self.data = self._execute_gpt_response()

    def _get_gpt_response(self) -> str:
        if not openai.api_key:
            raise ValueError("OPENAI_API_KEY environment variable is not set")
        
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that parses the user's prompt and returns the appropriate Python code to get data from Yahoo Finance. Return only Python code without markdown syntax or comments."},
                {"role": "user", "content": self.data_prompt}
            ]
        )
        print(response.choices[0].message.content)
        return response.choices[0].message.content
    
    def _execute_gpt_response(self) -> Dict[str, Any]:
        
        namespace: Dict[str, Any] = {"yf": yf, "DataFrame": DataFrame, "date": date}
        
        try:
            exec(self.pandas_intermediate_data, namespace)
        except Exception as e:
            raise RuntimeError(f"Error executing GPT response: {str(e)}")
        
        # Find the variable that holds the result (assuming it's the last assignment)
        result_vars = [var for var in namespace if not var.startswith("__") and var not in ["yf", "DataFrame", "date"]]
        if not result_vars:
            raise ValueError("No result found in GPT response")
        
        return namespace[result_vars[-1]]
    
    
    
    def _pandas_code_generate(self, data_prompt: str) -> str:
        if not openai.api_key:
            raise ValueError("OPENAI_API_KEY environment variable is not set")
        
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a data analyst and Python pandas expert. Generate concise, efficient pandas code to filter the dataframe. Return only the code, no markdown or comments."},
                {"role": "user", "content": f"Available columns: {', '.join(self.ticker_data.columns)}\nDataFrame: self.ticker_data\nTask: {data_prompt}\n Generate pandas code to filter the dataframe based on the task."}
            ]
        )

        result = response.choices[0].message.content

        print(f"Generated pandas code:\n{res}")
        
        return pandas_code


    def _pandas_code_execute(self) -> Dict[str, Any]:
        
        namespace: Dict[str, Any] = {"yf": yf, "DataFrame": DataFrame, "date": date, 'self': self}
        
        try:
            exec(self.pandas_code, namespace)
        except Exception as e:
            raise RuntimeError(f"Error executing pandas code: {str(e)}")
        
        # Find the variable that holds the result (assuming it's the last assignment)
        result_vars = [var for var in namespace if not var.startswith("__") and var not in ["yf", "DataFrame", "date"]]
        if not result_vars:
            raise ValueError("No result found in pandas code execution")
        print(namespace[result_vars[-1]])
        return namespace[result_vars[-1]]
    

    def __str__(self) -> str:
        return "Object to load data from Yahoo Finance"
    
    
    

inst = LoadData("I want to run analysis on all companies that have a market cap of over 100 billion dollars.")


#


/var/folders/v8/l9k84q0j5dvb69nc7wjtd5x80000gn/T/ipykernel_60191/541624650.py:16: DtypeWarning: Columns (1067,1376,1403,1453,1644,1930,1990,2137,2159,2178,2519,2583,2617,2672,2803,3031,3101,3193,3327,3532,3548,3631,3661,3685,3727,3787,3847,3852,3924,3980,3993,4000,4016,4066,4070,4092,4199,4205,4292,4357,4366,4371,4420,4532,4562,4597,5129,5187,5316,5455,5672,5708,5888,5902,6063,6163,6174,6212,6304,6571,6919,7116,7149,7152,7161,7168,7183,7190,7205,7208,7221,7234,7242,7247,7249,7250,7260,7261,7262,7264,7265,7266,7267,7268,7269,7270,7272,7273,7274,7275,7276,7277,7278,7282,7284,7286,7288,7289,7292,7294,7295,7296,7297,7298,7299,7300,7303,7304,7305,7306,7307,7310,7312,7313,7314,7317,7320,7323,7324,7325,7326,7327,7329,7330,7332,7336,7338,7341,7342,7345,7346,7347,7348,7349,7351,7353,7354,7355,7356,7358,7359,7361,7362,7363,7366,7367,7368,7369,7373,7374,7376,7377,7380,7381,7382,7386,7387,7388,7389,7392,7393,7394,7395,7400,7402,7403,7406,7407,7408,7409,7410,7412,7415,7417,7421,7423,7424,7426,7427,

Generated pandas code:
self.ticker_data[self.ticker_data['marketCap'] > 100000000000]


In [17]:
namespace: Dict[str, Any] = {"yf": yf, "DataFrame": DataFrame, "date": date}
        

exec(inst.pandas_code, namespace)

NameError: name 'self' is not defined